In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re
from nltk import ngrams
import pandas as pd
import json
from elasticsearch import Elasticsearch

stopwords_set = set(stopwords.words('english'))

In [3]:
es = Elasticsearch()
tknzr = TweetTokenizer()

emotions_to_parse = ["happy", "sad"]

In [19]:
tweets_df = pd.DataFrame(columns=['text', 'emotion'])

In [20]:
tweets_df.head(3)

,text,emotion


In [21]:
for emotion in emotions_to_parse:
    res = es.search(index="fetched_tweets", body={"query": {"match": { "full_text": "#" + emotion }}, "size": 10000})
    
    series = pd.Series()
    
    for msg in res['hits']['hits']:
        
#         print(msg)
#         break
        
        # Skip retweeted tweets
        if "retweeted_status" in msg['_source']:
            continue
        
        full_msg_text = msg['_source']['full_text'].replace("\n", " ").lower()
        
        tweets_df.loc[msg['_id']] = {
            'text': full_msg_text, 'emotion': emotion
        }
        
        

In [24]:
tweets_df[tweets_df['emotion'] == 'sad'].head(5)

,text,emotion
1136591706772582401,"for you, i’ll give a sad bye bye bye because y...",sad
1135665364111638535,@shanmharris misery loves company! so they can...,sad
1135703116802924549,@miafarrow darn those happy pills! #sad,sad
1136396141652787200,food and alcohol makes me happy #sad,sad
1135403471572627456,we are #sad #generation with #happy #pictures....,sad


In [87]:

skipped_ngrams = []
raw_data = []

for row in tweets_df.text.iteritems():
    tweet_id = row[0]
    text = row[1]
    
    n = 2
    for tokens in ngrams(text.split(), n):
        at_least_one_word_is_hashtag = False
        at_least_one_word_is_not_stopword = False
        contains_non_words = False

        # Filter ngrams a bit. We want to skip the following:
        #   - ngrams containing hashtags
        #   - ngrams containing all stopwords
        #   - ngrams that contain no actual words

        clean_words = []

        for word in tokens:
            if word[0] == "#":
                # Skip hashtags (not sure if this is a good idea though!)
                at_least_one_word_is_hashtag = True
                break

            # Some words end with ":" or "!" are treated as non words. I'd like to include those actually.
            # An example is: "very happy!". Basically, it's a valid ngram, but sign at the end of word
            # results in it being treated as invalid word
            word = re.sub(r'[!:,.]$', '', word)

            clean_words.append(word)

            if not re.match("^[a-zA-Z]+", word):
                contains_non_words = True
                break

            if re.match("^http[s]:\/\/", word):
                contains_non_words = True
                break

            if word not in stopwords_set:
                at_least_one_word_is_not_stopword = True

        # Skip ngrams that contain hashtags
        if at_least_one_word_is_hashtag or not at_least_one_word_is_not_stopword or contains_non_words:
    #             print "Skipping tuple {} because it contains hastahs".format(tokens)
            skipped_ngrams.append(ngram)
            continue

    #         # Skip ngrams that dont have at least ONE word that is not a stopword
    #         if not at_least_one_word_is_not_stopword:
    #             continue

        clean_word = " ".join(clean_words)

        raw_data.append([tweet_id, clean_word])
    
tweets_ngrams = pd.DataFrame(data=raw_data, columns=['tweet_id', 'ngram'])

In [88]:
merged_df = tweets_ngrams.merge(tweets_df, how='inner', left_on='tweet_id', right_index=True)

In [89]:
merged_df.head(3)

,tweet_id,ngram,text,emotion
0,1136544883064229888,happy kid,happy kid ❤ #happy https://t.co/6f3snnya52,happy
1,1136615042869338120,happy thursday,happy thursday. #nycsubway #meandyou #happy h...,happy
2,1136466744850038784,happy kev's,"@maddow ~looks comfy. #happy happy, kev's dad!🦑",happy


In [103]:
totals_df = merged_df.groupby(['emotion', 'ngram'])[['ngram']].count().rename(columns={'ngram': 'total'}).reset_index()
totals_df.head(5)


,emotion,ngram,total
0,happy,a bad,2
1,happy,a baggage,1
2,happy,a basketball,1
3,happy,a beautiful,7
4,happy,a belly,1


In [108]:
merged_df.groupby('emotion')['ngram'].count().reset_index().rename(columns={'ngram': 'total_ngrams'})

,emotion,total_ngrams
0,happy,10221
1,sad,39541


In [112]:
totals_df = totals_df.merge(
    merged_df.groupby('emotion')['ngram'].count().reset_index().rename(columns={'ngram': 'total_ngrams'}),
    how='inner',
    left_on='emotion',
    right_on='emotion'
)

In [113]:
totals_df.head(5)

,emotion,ngram,total,total_ngrams
0,happy,a bad,2,10221
1,happy,a baggage,1,10221
2,happy,a basketball,1,10221
3,happy,a beautiful,7,10221
4,happy,a belly,1,10221


In [114]:
totals_df['freq'] = totals_df['total'] / totals_df['total_ngrams']

In [117]:
totals_df[totals_df['emotion'] == 'happy'].sort_values(by='freq', ascending=False).head(30)

,emotion,ngram,total,total_ngrams,freq
5203,happy,photo courtesy,28,10221,0.002739
1613,happy,courtesy pinterest,28,10221,0.002739
6387,happy,thank you,23,10221,0.002250
7041,happy,to make,22,10221,0.002152
3215,happy,i love,16,10221,0.001565
6463,happy,the best,15,10221,0.001468
5980,happy,so much,13,10221,0.001272
2740,happy,good morning,11,10221,0.001076
1394,happy,check out,11,10221,0.001076
105,happy,a new,11,10221,0.001076
